In [5]:
#Seperating out accepted answers from the dataset to see if there's an increase the coherence score

#Step 1: Read CSV file into a dataframe

import pandas as pd

#Read in the files
df_1 = pd.read_csv('Posts1.csv')
df_2 = pd.read_csv('Posts2.csv')
df_3 = pd.read_csv('Posts3.csv')

#merge the three dataframes together
frames = [df_1, df_2, df_3]

df = pd.concat(frames, ignore_index=True)

In [40]:


#Drop any posts that aren't questions or answers
df = df.drop(df[(df.PostTypeId > 2)].index)

#Fill NaN values for AcceptedAnswerId with 0 - to prevent errors in other steps
df['AcceptedAnswerId']=df['AcceptedAnswerId'].fillna(0.0)

#Step 2: Group the dataframe rows by PostTypeId to seperate questions and answers, and create a list 
#        of acceptable answer Ids

#Group Question and Answer posts and aggregate the accepted answer Ids into lists 
#       - Answer posts  (2.0) should only have 0 values
gf = df.groupby('PostTypeId').agg({'AcceptedAnswerId': lambda x: list(x)})

#Step 3: Store AcceptedAnswerIds in a list where PostTypeId equals 1.0
# This represents the Ids for the answers that have been accepted by the user who posted the question
# 0.0 represents the question posts that had no accepted answer

#Move the acceptable answer ids list from a dataframe column into a list
answer_ids = [answers for answers in gf.at[1, 'AcceptedAnswerId']]

#Sort the list in ascending order
answer_ids.sort()

# Step 4: Drop all rows that are answers (PostTypeId = 2) that do not have an id in the 
#         accepted answer id list

#Drop a row if it is an answer and if it's Id is not on the acceptable answer id list
df = df.drop(df[(df.PostTypeId == 2) & (~df.Id.isin(answer_ids))].index)

# Step 5: Storing the results in a new csv file

df.to_csv('Posts-update.csv')